In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pkg_resources

import debvader

2021-12-10 12:26:56.109954: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pbs/throng/lsst/users/bbiswas/miniconda3/envs/debvader/lib/:
2021-12-10 12:26:56.112362: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pbs/throng/lsst/users/bbiswas/miniconda3/envs/debvader/lib/:
2021-12-10 12:26:56.112428: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


### Download and format data

Download dataset that will be used for training. It has been generated using the code in https://github.com/BastienArcelin/dc2_img_generation and the stamps are generated using the XXX function see notebook XXX. The size of the stamp is then fixed to 59x59 pixels.

In [2]:
data_folder_path = pkg_resources.resource_filename('debvader', "data/")
image_path = os.path.join(data_folder_path + 'dc2_imgs/imgs_dc2.npy')
images = np.load(image_path, mmap_mode = 'c')

In [3]:
images.shape

(10, 59, 59, 6)

Separate training data and labels to feed the VAE and deblender. You should be careful that the number of filter in the data for trainng correspond to the number of bands considered for the network (default is six).

In [4]:
training_data_vae = np.array((images[:5], images[:5]))
validation_data_vae = np.array((images[5:], images[5:]))

training_data_deblender = np.array((images[:5], images[:5]))
validation_data_deblender = np.array((images[5:], images[5:]))

### Train the VAE and deblender with architecture from the debvader paper (https://arxiv.org/abs/2005.12039)

In [5]:
hist_vae, hist_deblender, net = debvader.train.train_deblender("lsst",
                                                      from_survey = "dc2", 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender)

2021-12-10 12:27:06.999782: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pbs/throng/lsst/users/bbiswas/miniconda3/envs/debvader/lib/:
2021-12-10 12:27:06.999942: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-10 12:27:07.000064: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jns-bbiswas.cc.in2p3.fr): /proc/driver/nvidia/version does not exist
2021-12-10 12:27:07.003407: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-12-10 12:27:07.045817: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2399995000 Hz
2021-12-10 12:27:07.050170: I tensorflow/compiler/xla/service/service.cc

in cropping


2021-12-10 12:27:08.401285: W tensorflow/python/util/util.cc:319] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
VAE model
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 59, 59, 6)]       0         
_________________________________________________________________
model (Model)                (None, 560)               3741224   
_________________________________________________________________
multivariate_normal_tri_l (M ((None, 32), (None, 32))  0         
_________________________________________________________________
model_1 (Model)              (None, 59, 59, 6)         4577228   
Total params: 8,318,452
Trainable params: 8,318,440
Non-trainable params: 12
_________________________________________________________________
/pbs/throng/lsst/users/bbiswas/debvader/debvader/data/weights/dc2/not_normalised/

Start the training
Train on 5 samples, validate on 5 samples
Epoch

### Dataset with a different number of filters

The number of available filters is different for each survey. For example, five filters are available for DES data. To change that you should specify it in the train_deblender function, as well as if the channels appear last or first in the data array.

In [6]:
training_data_vae.shape

(2, 5, 59, 59, 6)

Here the channels appear last in the data array. So the channel_first option in the train_deblender function is set as False (default setting).

In [7]:
hist_vae, hist_deblender, net = debvader.train.train_deblender("des",
                                                      from_survey = "dc2", 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender,
                                                      nb_of_bands = 5,
                                                      channel_last = True)

in cropping
VAE model
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 59, 59, 5)]       0         
_________________________________________________________________
model_4 (Model)              (None, 560)               3740932   
_________________________________________________________________
multivariate_normal_tri_l_1  ((None, 32), (None, 32))  0         
_________________________________________________________________
model_5 (Model)              (None, 59, 59, 5)         4576650   
Total params: 8,317,582
Trainable params: 8,317,572
Non-trainable params: 10
_________________________________________________________________
The number of bands in the data does not correspond to the number of filters in the network. Correct this before starting again.


ValueError: 

We need to change the training data format. For example just train on the five fist channels.

In [8]:
training_data_vae_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_vae_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

training_data_deblender_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_deblender_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

Now try the training:

In [9]:
data_folder_path = pkg_resources.resource_filename('debvader', "data/")
path_output = os.path.join(data_folder_path, 'weights/dc2/not_normalised/')
latest = tf.train.latest_checkpoint(path_output)

Warning: we cannot use the weights from the network trained on DC2 as the data has six channels in that case. We need to set the from_survey option to None.

In [10]:
hist_vae, hist_deblender, net = debvader.train.train_deblender("des",
                                                      from_survey = None, 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae_deslike, 
                                                      validation_data_vae = validation_data_vae_deslike, 
                                                      training_data_deblender = training_data_deblender_deslike, 
                                                      validation_data_deblender = validation_data_deblender_deslike,
                                                      nb_of_bands = 5,
                                                      channel_last = True)

in cropping
VAE model
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 59, 59, 5)]       0         
_________________________________________________________________
model_8 (Model)              (None, 560)               3740932   
_________________________________________________________________
multivariate_normal_tri_l_2  ((None, 32), (None, 32))  0         
_________________________________________________________________
model_9 (Model)              (None, 59, 59, 5)         4576650   
Total params: 8,317,582
Trainable params: 8,317,572
Non-trainable params: 10
_________________________________________________________________

Start the training
Train on 5 samples, validate on 5 samples
Epoch 1/2

Epoch 00001: val_mse improved from inf to 0.09295, saving model to /pbs/throng/lsst/users/bbiswas/debvader/debvader/data/weights/des/vae/val_mse/weigh